* Name: `dissipation-theory--Study-18.ipynb`
* Author: John A. Marohn
* Date: 2024-06-02
* Continued from: `dissipation-theory--Study-17.ipynb`
* Continued to: ---
* Summary: Turn the BLDS fitting code in `dissipation-theory--Study-17.ipynb` into functions that can be looped over data.

In [1]:
from dissipationtheory.constants import ureg
from dissipationtheory.dissipation import blds_perpendicular_jit
from dissipationtheory.dissipation import CantileverModelJit, SampleModel1Jit
from dissipationtheory.dissipation import theta1norm_jit
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
import matplotlib.pylab as plt

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 12}

plt.rc('font', **font)
plt.rcParams['figure.figsize'] = 3.25, 3.5

In [3]:
THIS = 'dissipation-theory--Study-18--'